In [1]:
#-*- coding:utf-8 -*-
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import minmax_scale
import requests
import pymysql
import random
from tqdm import tqdm
import time
import math
import joblib

tierList = ['IRON','BRONZE','SILVER','GOLD','PLATINUM','EMERALD','DIAMOND']
# tierList = ['DIAMOND']
teamPositionList = ['TOP', 'JUNGLE', 'MIDDLE','BOTTOM','UTILITY']
championNameList = []

#########################
def connect_mysql(db='mydb'):
    conn = pymysql.connect(host='svc.sel4.cloudtype.app', port=32509,
                        user='root', password='1234',
                        db=db, charset='utf8')
    return conn

queueId = str(420) #큐아이디
limit_value = str(1000) # 리미트값
############################

conn = connect_mysql()
cursor = conn.cursor()
query = "SELECT champion_name FROM champListT;" #총 챔피언 리스트
cursor.execute(query)
result = cursor.fetchall()
data = list(result)
for i in range(len(data)):

    championNameList.append(data[i][0])   
        
for tier in tierList:
    for teamPosition in teamPositionList:
        

        query = "select win,gameDuration,kda,totalDamageDealtToChampions,goldEarned,championName from riottvT where tier = '" +tier+ "' and teamPosition = '" +teamPosition+ "' and queueId = '" +queueId+ "'" #질때의 kda
        cursor.execute(query)
        result = cursor.fetchall()
        data = list(result)
        print(len(data))
          
        for championName1 in championNameList:
            testList = []
            for i in range(len(data)): 
                win,gameDuration,kda,totalDamageDealtToChampions,goldEarned,championName = data[i]
                # print(championName)
                if championName == championName1:
                    testList.append(data[i])

            if len(testList)>100 :
                print(len(testList))
                lose_gameDuration_List = []
                lose_kda_List = []
                lose_Mean_totalDamageDealtToChampions = []
                lose_Mean_goldEarned = []

                win_gameDuration_List = []
                win_kda_List = []
                win_Mean_totalDamageDealtToChampions = []
                win_Mean_goldEarned = []

                for e in testList:
                    win,gameDuration,kda,totalDamageDealtToChampions,goldEarned,championName = e
                    if win == 'False':
                        lose_gameDuration_List.append(gameDuration)
                        lose_kda_List.append(kda)
                        lose_Mean_totalDamageDealtToChampions.append(int(totalDamageDealtToChampions/gameDuration))
                        lose_Mean_goldEarned.append(int(goldEarned/gameDuration))
                    elif win == 'True' :
                        win_gameDuration_List.append(gameDuration)
                        win_kda_List.append(kda)
                        win_Mean_totalDamageDealtToChampions.append(int(totalDamageDealtToChampions/gameDuration))
                        win_Mean_goldEarned.append(int(goldEarned/gameDuration))

                allkda = win_kda_List+lose_kda_List
                Mean_totalDamageDealtToChampions = win_Mean_totalDamageDealtToChampions + lose_Mean_totalDamageDealtToChampions
                Mean_goldEarned = win_Mean_goldEarned + lose_Mean_goldEarned

                tier_data=[[t,g,k]for t,g,k in zip(Mean_totalDamageDealtToChampions,Mean_goldEarned,allkda)]

                tier_target=[1]*len(win_kda_List)+[0]*len(lose_kda_List)
                #z-정규화
                mean = np.mean(tier_data, axis=0)
                std = np.std(tier_data, axis=0)
                train_scaled = (tier_data - mean) / std

                dataset = {'data' : train_scaled, 'target' : tier_target}

            # 입력 데이터와 타깃을 준비합니다.
                X, y = dataset['data'], dataset['target']

                # K 최근접이웃 모델 객체를 만듭니다.
                model = KNeighborsClassifier(n_neighbors=3)
                
            # K 최근접 이웃 모델에 입력 데이터와 타깃을 입력하고 학습시킨다.
                model.fit(X,y)

                # 학습시킨 모델을 현재 경로에 knn_model.pkl 파일로 저장합니다.
                joblib.dump(model, './'+queueId+'_'+tier+'_'+teamPosition+ '_' + championName1+'.pkl')
                
                print(queueId+'_'+tier+'_'+teamPosition+ '_' + championName1+'저장완료')

            else:
                print(queueId+'_'+tier+'_'+teamPosition+ '_' + championName1+'패스')
            
conn.close()            
                        


18948
420_IRON_TOP_Annie패스
124
420_IRON_TOP_Olaf저장완료
420_IRON_TOP_Galio패스
128
420_IRON_TOP_TwistedFate저장완료
420_IRON_TOP_XinZhao패스
145
420_IRON_TOP_Urgot저장완료
420_IRON_TOP_Leblanc패스
420_IRON_TOP_Vladimir패스
420_IRON_TOP_Fiddlesticks패스
197
420_IRON_TOP_Kayle저장완료
420_IRON_TOP_MasterYi패스
420_IRON_TOP_Alistar패스
420_IRON_TOP_Ryze패스
137
420_IRON_TOP_Sion저장완료
420_IRON_TOP_Sivir패스
420_IRON_TOP_Soraka패스
716
420_IRON_TOP_Teemo저장완료
420_IRON_TOP_Tristana패스
136
420_IRON_TOP_Warwick저장완료
420_IRON_TOP_Nunu패스
420_IRON_TOP_MissFortune패스
420_IRON_TOP_Ashe패스
195
420_IRON_TOP_Tryndamere저장완료
502
420_IRON_TOP_Jax저장완료
420_IRON_TOP_Morgana패스
420_IRON_TOP_Zilean패스
420_IRON_TOP_Singed패스
420_IRON_TOP_Evelynn패스
420_IRON_TOP_Twitch패스
420_IRON_TOP_Karthus패스
420_IRON_TOP_Chogath패스
420_IRON_TOP_Amumu패스
420_IRON_TOP_Rammus패스
420_IRON_TOP_Anivia패스
420_IRON_TOP_Shaco패스
322
420_IRON_TOP_DrMundo저장완료
420_IRON_TOP_Sona패스
420_IRON_TOP_Kassadin패스
336
420_IRON_TOP_Irelia저장완료
420_IRON_TOP_Janna패스
128
420_IRON_TOP_Gangplank저장완료
420_

In [13]:
#-*- coding:utf-8 -*-
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import minmax_scale
import requests
import pymysql
import random
from tqdm import tqdm
import time
import math
import joblib

#받은 데이터

data = sys.argv[1:]
queueId = 420
key = '1'
tier = 'GOLD'
teamPosition = 'TOP'
gameDuration = 20
kda = 5
totalDamageDealtToChampions = 22300
goldEarned = 20000
championName = 'Olaf'



try:
    ####
    totalDamageDealtToChampions = 2000
    goldEarned = 900
    teamPosition = 'TOP'
    championName = 'Olaf' #캐릭터
    tier_my = [totalDamageDealtToChampions,goldEarned,kda]
    str1 = f"./aiModel420/{queueId}_{tier}_{teamPosition}_{championName}.pkl"
    # print(str1)

    loaded_model = joblib.load(str1)
   
    #z-정규화
    mean = np.mean(tier_my, axis=0)
    std = np.std(tier_my, axis=0)

    new = (tier_my - mean) / std

    trans={1:'승', 0:'패'}
    a = trans[loaded_model.predict([new])[0]]

    data5 = {key:a  , '구간' : tier , '캐릭' : championName, "key" : a}  
    json_string = json.dumps(data5)
    print(data5)
except Exception as e:
    data5 = {key:"에러" , '구간' : tier , '캐릭' : championName, "key" : "에러" }  
    json_string = json.dumps(data5)
    print(data5)

{'1': '패', '구간': 'GOLD', '캐릭': 'Olaf', 'key': '패'}


In [11]:
str1

'./aiModel420/420_GOLD_TOP_Olaf.pkl'